In [18]:
import numpy as np
import networkx as nx
import copy
import pickle
import sys
import time
sys.path.append("/Users/ziniuw/Desktop/research/Learned_QO/CE_scheme")
from BayesCard.Evaluation.utils import parse_query
from Schemas.stats.schema import gen_stats_light_schema
from Join_scheme.data_prepare import identify_key_values, process_stats_data

In [48]:
data_path = "/Users/ziniuw/Desktop/past_research/End-to-End-CardEst-Benchmark/datasets/stats_simplified/{}.csv"
schema = gen_stats_light_schema(data_path)
all_keys, equivalent_keys = identify_key_values(schema)
print(equivalent_keys)

{'posts.Id': {'postLinks.RelatedPostId', 'postHistory.PostId', 'tags.ExcerptPostId', 'postLinks.PostId', 'comments.PostId', 'votes.PostId', 'posts.Id'}, 'users.Id': {'badges.UserId', 'users.Id', 'votes.UserId', 'postHistory.UserId', 'posts.OwnerUserId', 'comments.UserId'}}


In [49]:
model_path = "/Users/ziniuw/Desktop/research/Learned_QO/CC_model/CE_scheme_models/stats/model_200.pkl"
with open(model_path, "rb") as f:
    bound_ensemble = pickle.load(f)

In [50]:
bound_ensemble.bns

{'badges': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7fb50ab08210>,
 'votes': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7fb4e911e6d0>,
 'postHistory': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7fb4f004bc10>,
 'posts': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7fb50c50dd90>,
 'users': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7fb4f004b3d0>,
 'comments': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7fb4f0083390>,
 'postLinks': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7fb4f0083950>,
 'tags': <BayesCard.Models.Bayescard_BN.Bayescard_BN at 0x7fb4f0083f90>}

In [19]:
true_card = 17849233970
query57 = "SELECT COUNT(*) FROM posts as p, postLinks as pl, postHistory as ph, votes as v, badges as b, users as u WHERE p.Id = pl.RelatedPostId AND u.Id = p.OwnerUserId AND u.Id = b.UserId AND u.Id = ph.UserId AND u.Id = v.UserId AND p.CommentCount>=0 AND p.CommentCount<=13 AND ph.PostHistoryTypeId=5 AND ph.CreationDate<='2014-08-13 09:20:10'::timestamp AND v.CreationDate>='2010-07-19 00:00:00'::timestamp AND b.Date<='2014-09-09 10:24:35'::timestamp AND u.Views>=0 AND u.DownVotes>=0 AND u.CreationDate>='2010-08-04 16:59:53'::timestamp AND u.CreationDate<='2014-07-22 15:15:22'::timestamp;"
t = time.time()
tables_all, table_queries, join_cond, join_keys = bound_ensemble.parse_query_simple(query57)
print(time.time()-t)

0.0022041797637939453


In [25]:
t = time.time()
a = bound_ensemble.get_all_id_conidtional_distribution(table_queries, join_keys)
print(time.time()-t)
for table in a:
    print(a[table][0], np.sum(a[table][1]))


0.04527783393859863
['posts.OwnerUserId', 'posts.Id'] 89895.23949291356
['postLinks.RelatedPostId'] 11102.0
['users.Id'] 26725.90090846615
['badges.UserId'] 78519.0
['postHistory.UserId'] 66103.19701428554
['votes.UserId'] 34765.435993975894


In [26]:
print("join keys:", join_keys)

join keys: {'posts': {'posts.Id', 'posts.OwnerUserId'}, 'postLinks': {'postLinks.RelatedPostId'}, 'users': {'users.Id'}, 'badges': {'badges.UserId'}, 'postHistory': {'postHistory.UserId'}, 'votes': {'votes.UserId'}}


In [30]:
from Join_scheme.join_graph import process_condition, get_join_hyper_graph
equivalent_group = get_join_hyper_graph(join_keys, bound_ensemble.equivalent_keys)

In [75]:
import numpy as np
import copy

from Join_scheme.join_graph import process_condition, get_join_hyper_graph
from Join_scheme.data_prepare import identify_key_values
from BayesCard.Evaluation.cardinality_estimation import timestamp_transorform, construct_table_query


class Factor:
    """
    This the class defines a multidimensional conditional probability.
    """
    def __init__(self, variables, pdfs, equivalent_variables=[]):
        self.variables = variables
        self.equivalent_variables = equivalent_variables
        self.pdfs = pdfs
        self.cardinalities = dict()
        for i, var in enumerate(self.variables):
            self.cardinalities[var] = pdfs.shape[i]
            if len(equivalent_variables) != 0:
                self.cardinalities[equivalent_variables[i]] = pdfs.shape[i]


class Bound_ensemble:
    """
    This the class where we store all the trained models and perform inference on the bound.
    """
    def __init__(self, bns, table_buckets, schema):
        self.bns = bns
        self.table_buckets = table_buckets
        self.schema = schema
        self.all_keys, self.equivalent_keys = identify_key_values(schema)

    def parse_query_simple(self, query):
        """
        If your selection query contains no aggregation and nested sub-queries, you can use this function to parse a
        join query. Otherwise, use parse_query function.
        """
        query = query.replace(" where ", " WHERE ")
        query = query.replace(" from ", " FROM ")
        query = query.replace(" and ", " AND ")
        query = query.split(";")[0]
        query = query.strip()
        tables_all = {}
        join_cond = []
        table_query = {}
        join_keys = {}
        tables_str = query.split(" WHERE ")[0].split(" FROM ")[-1]
        for table_str in tables_str.split(","):
            table_str = table_str.strip()
            if " as " in table_str:
                tables_all[table_str.split(" as ")[-1]] = table_str.split(" as ")[0]
            else:
                tables_all[table_str.split(" ")[-1]] = table_str.split(" ")[0]

        # processing conditions
        conditions = query.split(" WHERE ")[-1].split(" AND ")
        for cond in conditions:
            table, cond, join, join_key = process_condition(cond, tables_all)
            if not join:
                attr = cond[0]
                op = cond[1]
                value = cond[2]
                if "Date" in attr:
                    assert "::timestamp" in value
                    value = timestamp_transorform(value.strip().split("::timestamp")[0])
                if table not in table_query:
                    table_query[table] = dict()
                construct_table_query(self.bns[table], table_query[table], attr, op, value)
            else:
                join_cond.append(cond)
                for tab in join_key:
                    if tab in join_keys:
                        join_keys[tab].add(join_key[tab])
                    else:
                        join_keys[tab] = set([join_key[tab]])

        return tables_all, table_query, join_cond, join_keys

    def get_all_id_conidtional_distribution(self, table_queries, join_keys, equivalent_group):
        res = dict()
        for table in join_keys:
            key_attrs = list(join_keys[table])
            if table in table_queries:
                table_query = table_queries[table]
            else:
                table_query = {}
            id_attrs, probs = self.bns[table].query_id_prob(table_query, key_attrs)
            new_id_attrs = []
            for K in id_attrs:
                for PK in equivalent_group:
                    if K in equivalent_group[PK]:
                        new_id_attrs.append(PK)
            assert len(new_id_attrs) == len(id_attrs)
            res[table] = Factor(id_attrs, probs, new_id_attrs)
        return res

    def eliminate_one_key_group_general(self, tables, key_group, factors):
        rest_groups = dict()
        rest_group_tables = dict()
        for table in tables:
            assert key_group in factors[table].equivalent_variables
            temp = copy.deepcopy(factors[table].equivalent_variables)
            temp.remove(key_group)
            for keys in temp:
                if keys in rest_groups:
                    assert factors[table].cardinalities[keys] == rest_groups[keys]
                    rest_group_tables[keys].append(table)
                else:
                    rest_groups[keys] = factors[table].cardinalities[keys]
                    rest_group_tables[keys] = [table]

        new_factor_variables = []
        new_factor_cardinalities = []
        for key in rest_groups:
            new_factor_variables.append(key)
            new_factor_cardinalities.append(rest_groups[key])
        new_factor_pdf = np.zeros(tuple(new_factor_cardinalities))

    def eliminate_one_key_group(self, tables, key_group, factors, relevant_keys):
        """This version only supports 2D distributions"""
        rest_group = None
        rest_group_cardinalty = 0
        eliminated_tables = []
        rest_group_tables = []
        for table in tables:
            assert key_group in factors[table].equivalent_variables
            temp = copy.deepcopy(factors[table].equivalent_variables)
            temp.remove(key_group)
            if len(temp) == 0:
                eliminated_tables.append(table)
            for key in temp:
                if rest_group:
                    assert factors[table].cardinalities[key] == rest_group_cardinalty
                    rest_group_tables.append(table)
                else:
                    rest_group = key
                    rest_group_cardinalty = factors[table].cardinalities[key]
                    rest_group_tables = [table]

        all_probs_eliminated = []
        all_modes_eliminated = []
        for table in eliminated_tables:
            bin_modes = self.table_buckets[table].oned_bin_modes[relevant_keys[key_group][table]]
            all_probs_eliminated.append(factors[table].pdfs)
            all_modes_eliminated.append(np.minimum(bin_modes, factors[table].pdfs))

        if rest_group:
            new_factor_pdf = np.zeros(rest_group_cardinalty)
        else:
            return self.compute_bound_oned(all_probs_eliminated, all_modes_eliminated)

        for i in range(rest_group_cardinalty):
            for table in rest_group_tables:
                idx_f = factors[table].equivalent_variables.index(key_group)
                idx_b = self.table_buckets[table].id_attributes.index(relevant_keys[key_group][table])
                bin_modes = self.table_buckets[table].twod_bin_modes[relevant_keys[key_group][table]]
                if idx_f == 0 and idx_b == 0:
                    all_probs_eliminated.append(factors[table].pdfs[:, i])
                    all_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[:, i]))
                elif idx_f == 0 and idx_b == 1:
                    all_probs_eliminated.append(factors[table].pdfs[:, i])
                    all_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[:, i]))
                elif idx_f == 1 and idx_b == 0:
                    all_probs_eliminated.append(factors[table].pdfs[i, :])
                    all_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[i, :]))
                else:
                    all_probs_eliminated.append(factors[table].pdfs[i, :])
                    all_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[i, :]))
            new_factor_pdf[i] = self.compute_bound_oned(all_probs_eliminated, all_modes_eliminated)

        for table in rest_group_tables:
            factors[table] = Factor([rest_group], new_factor_pdf, [rest_group])

        return None

    def compute_bound_oned(self, all_probs, all_modes):
        all_probs = np.stack(all_probs, axis=0)
        all_modes = np.stack(all_modes, axis=0)
        multiplier = np.prod(all_modes, axis=0)
        non_zero_idx = np.where(multiplier != 0)[0]
        min_number = np.amin(all_probs[:, non_zero_idx]/all_modes[:, non_zero_idx], axis=0)
        multiplier[non_zero_idx] = multiplier[non_zero_idx] * min_number
        return np.sum(multiplier)

    def get_optimal_elimination_order(self, equivalent_group, join_keys, factors):
        cardinalities = dict()
        lengths = dict()
        tables_involved = dict()
        relevant_keys = dict()
        for group in equivalent_group:
            relevant_keys[group] = dict()
            lengths[group] = len(equivalent_group[group])
            cardinalities[group] = []
            tables_involved[group] = set([])
            for keys in equivalent_group[group]:
                for table in join_keys:
                    if keys in join_keys[table]:
                        cardinalities[group].append(len(join_keys[table]))
                        tables_involved[group].add(table)
                        variables = factors[table].variables
                        variables[variables.index(keys)] = group
                        factors[table].variables = variables
                        relevant_keys[group][table] = keys
                        break
            cardinalities[group] = np.asarray(cardinalities[group])

        optimal_order = list(equivalent_group.keys())
        for i in range(len(optimal_order)):
            min_idx = i
            for j in range(i+1, len(optimal_order)):
                min_group = optimal_order[min_idx]
                curr_group = optimal_order[j]
                if np.max(cardinalities[curr_group]) < np.max(cardinalities[min_group]):
                    min_idx = j
                else:
                    min_max_tables = np.max(cardinalities[min_group])
                    min_num_max_tables = len(np.where(cardinalities[min_group] == min_max_tables)[0])
                    curr_max_tables = np.max(cardinalities[curr_group])
                    curr_num_max_tables = len(np.where(cardinalities[curr_group] == curr_max_tables)[0])
                    if curr_num_max_tables < min_num_max_tables:
                        min_idx = j
                    elif lengths[curr_group] < lengths[min_group]:
                        min_idx = j
            optimal_order[i], optimal_order[min_idx] = optimal_order[min_idx], optimal_order[i]
        return optimal_order, tables_involved, relevant_keys

    def get_cardinality_bound(self, query_str):
        tables_all, table_queries, join_cond, join_keys = self.parse_query_simple(query_str)
        equivalent_group = get_join_hyper_graph(join_keys, self.equivalent_keys)
        conditional_factors = self.get_all_id_conidtional_distribution(table_queries, join_keys, equivalent_group)
        optimal_order, tables_involved, relevant_keys = self.get_optimal_elimination_order(equivalent_group, join_keys,
                                                                            conditional_factors)

        for key_group in optimal_order:
            tables = tables_involved[key_group]
            res = self.eliminate_one_key_group(tables, key_group, conditional_factors, relevant_keys)
        return res

In [76]:
new_BE = Bound_ensemble(bound_ensemble.bns, bound_ensemble.table_buckets, bound_ensemble.schema)

In [78]:
query_str = "SELECT COUNT(*) FROM users as u, comments as c, posts as p WHERE p.OwnerUserId = u.Id AND p.Id = c.PostId AND u.UpVotes>=0 AND u.CreationDate>='2010-08-21 21:27:38'::timestamp AND c.CreationDate>='2010-07-21 11:05:37'::timestamp AND c.CreationDate<='2014-08-25 17:59:25'::timestamp"
t = time.time()
res = new_BE.get_cardinality_bound(query57)
print(time.time() - t)
print(res)

0.06695389747619629
1167089949644.402


In [79]:
query_file = "/Users/ziniuw/Desktop/past_research/End-to-End-CardEst-Benchmark/workloads/stats_CEB/sub_plan_queries/stats_CEB_sub_queries.sql"
with open(query_file, "r") as f:
    queries = f.readlines()


In [83]:
qerror = []
latency = []
for i, query_str in enumerate(queries):
    query = query_str.split("||")[0][:-1]
    print("========================")
    true_card = int(query_str.split("||")[-1])
    t = time.time()
    res = new_BE.get_cardinality_bound(query)
    latency.append(time.time() - t)
    qerror.append(res/true_card)
    print(f"estimating query {i}: predicted {res}, true_card {true_card}, qerror {res/true_card}, latency {time.time() - t}")

estimating query 0: predicted 79851.0, true_card 79851, qerror 1.0, latency 0.011759757995605469
estimating query 1: predicted 12028778.0, true_card 10220614, qerror 1.1769134417951799, latency 0.009178876876831055
estimating query 2: predicted 3935962.0, true_card 1458075, qerror 2.6994235550297483, latency 0.004566669464111328
estimating query 3: predicted 4017842.348968956, true_card 1709781, qerror 2.3499163629546453, latency 0.0069620609283447266
estimating query 4: predicted 10505630.0, true_card 7491903, qerror 1.4022645514764407, latency 0.007244110107421875
estimating query 5: predicted 1028558.2624406518, true_card 428612, qerror 2.399742103442395, latency 0.010093927383422852
estimating query 6: predicted 59710469.99099071, true_card 55900138, qerror 1.068163194713235, latency 0.0072231292724609375
estimating query 7: predicted 11102.0, true_card 10972, qerror 1.0118483412322274, latency 0.012944936752319336
estimating query 8: predicted 60664815.0, true_card 699302, qerror 

estimating query 89: predicted 150350.84035063183, true_card 8528, qerror 17.63025801484895, latency 0.01968693733215332
estimating query 90: predicted 269613.44018954964, true_card 25033, qerror 10.77032078414691, latency 0.012313127517700195
estimating query 91: predicted 39571365.34295467, true_card 36980622, qerror 1.0700567811691937, latency 0.005853891372680664
estimating query 92: predicted 10307870.8396423, true_card 242937, qerror 42.430221990237385, latency 0.010247945785522461
estimating query 93: predicted 71300.55752706007, true_card 42543, qerror 1.6759644953825557, latency 0.005567073822021484
estimating query 94: predicted 3609057.6253296044, true_card 84481, qerror 42.72034688663255, latency 0.011057853698730469
estimating query 95: predicted 55838.26126769599, true_card 40496, qerror 1.3788586840106676, latency 0.005669116973876953
estimating query 96: predicted 27772.372717454848, true_card 244, qerror 113.82119966170019, latency 0.009667158126831055
estimating query

estimating query 148: predicted 3329161.233683795, true_card 221701, qerror 15.016446627141038, latency 0.03909802436828613
estimating query 149: predicted 2112401.1900483957, true_card 425598, qerror 4.963371984944468, latency 0.03655385971069336
estimating query 150: predicted 689159551.4626155, true_card 19402569, qerror 35.518984700562875, latency 0.037870168685913086
estimating query 151: predicted 263651356.0, true_card 263105194, qerror 1.0020758313117908, latency 0.0029289722442626953
estimating query 152: predicted 618124.6749487225, true_card 505601, qerror 1.2225542966661902, latency 0.03168368339538574
estimating query 153: predicted 55752.108009266754, true_card 19556, qerror 2.8508952755812413, latency 0.008128166198730469
estimating query 154: predicted 527110.0189160528, true_card 593512, qerror 0.8881202383710064, latency 0.030715227127075195
estimating query 155: predicted 73195.85327353755, true_card 62790, qerror 1.1657246898158553, latency 0.007537841796875
estimat

estimating query 214: predicted 11783633.756591402, true_card 9976404, qerror 1.1811504181858916, latency 0.00467681884765625
estimating query 215: predicted 971560.1618962524, true_card 443440, qerror 2.190961938247006, latency 0.003718137741088867
estimating query 216: predicted 268208434.99273983, true_card 105742741, qerror 2.536424084115049, latency 0.005894184112548828
estimating query 217: predicted 75991438.51316096, true_card 69114343, qerror 1.099503159758908, latency 0.005507946014404297
estimating query 218: predicted 11962012.508479482, true_card 10235746, qerror 1.1686507762579768, latency 0.00380706787109375
estimating query 219: predicted 66064.89245114151, true_card 48758, qerror 1.3549549294708871, latency 0.009273052215576172
estimating query 220: predicted 6881899.624729484, true_card 6011297, qerror 1.1448277509378566, latency 0.002850055694580078
estimating query 221: predicted 24217.353291058287, true_card 16892, qerror 1.4336581394185584, latency 0.0085978507995

estimating query 295: predicted 289761.464591593, true_card 10594, qerror 27.35146918931405, latency 0.006127119064331055
estimating query 296: predicted 3275990.7497134022, true_card 160411, qerror 20.422481935237624, latency 0.0071773529052734375
estimating query 297: predicted 680901.8321165385, true_card 147586, qerror 4.613593647883529, latency 0.010164976119995117
estimating query 298: predicted 6099498.9685763065, true_card 194687, qerror 31.329770187923724, latency 0.005942106246948242
estimating query 299: predicted 1649519.0781350345, true_card 262343, qerror 6.287642811643667, latency 0.009099960327148438
estimating query 300: predicted 18560620.9952061, true_card 3918357, qerror 4.736837658030164, latency 0.010230064392089844
estimating query 301: predicted 192859.14731373437, true_card 19015, qerror 10.142474221074645, latency 0.007452726364135742
estimating query 302: predicted 41633.82468141135, true_card 18097, qerror 2.300592622059532, latency 0.011001110076904297
esti

...

estimating query 2295: predicted 19922139.0, true_card 69119371, qerror 0.28822801353328287, latency 0.020907163619995117
estimating query 2296: predicted 7477673.047201302, true_card 679830, qerror 10.999327842550787, latency 0.011101961135864258
estimating query 2297: predicted 8993841.779900413, true_card 71466602, qerror 0.12584678056892104, latency 0.00971221923828125
estimating query 2298: predicted 8366030.471252398, true_card 728017, qerror 11.491531751665686, latency 0.009252071380615234
estimating query 2299: predicted 19669719.702570852, true_card 15683189, qerror 1.2541913320416436, latency 0.018482208251953125
estimating query 2300: predicted 108230064.77652158, true_card 9265894, qerror 11.68047732647509, latency 0.0037200450897216797
estimating query 2301: predicted 5394705.5239358535, true_card 23808461, qerror 0.22658774642913096, latency 0.009329080581665039
estimating query 2302: predicted 5394705.5239358535, true_card 205841, qerror 26.20811948997456, latency 0.0088

estimating query 2358: predicted 19805565.048099007, true_card 46537849, qerror 0.4255797264738, latency 0.04548811912536621
estimating query 2359: predicted 8700647.054623112, true_card 574721, qerror 15.138905755354532, latency 0.037850141525268555
estimating query 2360: predicted 11962012.508479482, true_card 10225026, qerror 1.1698759991886065, latency 0.0036208629608154297
estimating query 2361: predicted 212578.01998091326, true_card 68389, qerror 3.108365672563033, latency 0.005549192428588867
estimating query 2362: predicted 338824.073223793, true_card 300737, qerror 1.1266457842692885, latency 0.03121209144592285
estimating query 2363: predicted 1387797.1799698796, true_card 928622, qerror 1.4944694180946387, latency 0.03229713439941406
estimating query 2364: predicted 5640572.703661164, true_card 2368493, qerror 2.3815027967830873, latency 0.03805899620056152
estimating query 2365: predicted 5673918.744932784, true_card 121625, qerror 46.65092493264365, latency 0.007776021957

estimating query 2415: predicted 200042.8461217382, true_card 71359, qerror 2.8033302894062166, latency 0.003735065460205078
estimating query 2416: predicted 99639.8101170712, true_card 55484, qerror 1.7958296106457934, latency 0.008386850357055664
estimating query 2417: predicted 200328.23200344955, true_card 65334, qerror 3.0662171611021756, latency 0.0033397674560546875
estimating query 2418: predicted 321451.9993164329, true_card 164951, qerror 1.94877266167791, latency 0.008401155471801758
estimating query 2419: predicted 1442147.141777229, true_card 987389, qerror 1.4605663439406646, latency 0.009019136428833008
estimating query 2420: predicted 75934.1176165118, true_card 75910, qerror 1.0003177132987986, latency 0.0022649765014648438
estimating query 2421: predicted 263546.9686277766, true_card 16583, qerror 15.892598964468226, latency 0.0039520263671875
estimating query 2422: predicted 3373477.2827555174, true_card 185793, qerror 18.15718182469478, latency 0.005192995071411133


estimating query 2478: predicted 5394705.5239358535, true_card 12492036, qerror 0.43185158319555383, latency 0.012410879135131836
estimating query 2479: predicted 6110707.296048014, true_card 21582838, qerror 0.2831280712966485, latency 0.016920804977416992
estimating query 2480: predicted 35313884.00197089, true_card 96498890, qerror 0.36595119386317176, latency 0.019054174423217773
estimating query 2481: predicted 108793440.07878518, true_card 524182774, qerror 0.20754867476584643, latency 0.0171658992767334
estimating query 2482: predicted 108793440.07878518, true_card 155696936, qerror 0.6987513233965322, latency 0.01819586753845215
estimating query 2483: predicted 174305.0, true_card 90677, qerror 1.9222625362550592, latency 0.002062082290649414
estimating query 2484: predicted 3620673.0, true_card 704085, qerror 5.142380536440912, latency 0.002396106719970703
estimating query 2485: predicted 946691.0, true_card 865125, qerror 1.0942823291431873, latency 0.0007131099700927734
esti

estimating query 2552: predicted 211932.87084669137, true_card 12124, qerror 17.48044134334307, latency 0.013847827911376953
estimating query 2553: predicted 51897.41723176858, true_card 60963, qerror 0.8512936901361249, latency 0.022923946380615234
estimating query 2554: predicted 292844.8655594016, true_card 172157, qerror 1.7010337398967312, latency 0.01779937744140625
estimating query 2555: predicted 31404994.267896555, true_card 438690, qerror 71.58812434269429, latency 0.011406898498535156
estimating query 2556: predicted 6537629.0681315735, true_card 50824, qerror 128.63271423208667, latency 0.016010046005249023
estimating query 2557: predicted 1420595.922974716, true_card 249157, qerror 5.701609519197598, latency 0.017090797424316406
estimating query 2558: predicted 8465544.953835867, true_card 659522, qerror 12.835879551911638, latency 0.01776289939880371
estimating query 2559: predicted 1020322.1051143908, true_card 87188, qerror 11.702552015350632, latency 0.0212039947509765

In [84]:
qerror = np.asarray(qerror)

In [87]:
temp_qerror = copy.deepcopy(qerror)
temp_qerror[temp_qerror < 1] = 1/temp_qerror[temp_qerror < 1]

In [88]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(temp_qerror, i)}")

q-error 50% percentile is 3.3975305522594246
q-error 90% percentile is 45.1345082867916
q-error 95% percentile is 128.56434745297838
q-error 99% percentile is 1347.9017899591543
q-error 100% percentile is 436888.8917196179


In [90]:
np.where(qerror == np.max(qerror))

(array([362]),)